<a href="https://colab.research.google.com/github/Eduardismund/lifeline_ai/blob/main/lifeline_ai_gpt_oss_20b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
# We're installing the latest Torch, Triton, OpenAI's Triton kernels, Transformers and Unsloth!
!pip install --upgrade -qqq uv
try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
except: get_numpy = "numpy"
!uv pip install -qqq \
    "torch>=2.8.0" "triton>=3.4.0" {get_numpy} torchvision bitsandbytes \
    "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
    "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
    git+https://github.com/huggingface/transformers \
    git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels

In [ ]:
!find / -name "*.ipynb" 2>/dev/null

/opt/nvidia/nsight-compute/2024.2.1/host/target-linux-x64/python/packages/nsys_recipe/recipes/diff/diff.ipynb
/opt/nvidia/nsight-compute/2024.2.1/host/target-linux-x64/python/packages/nsys_recipe/recipes/network_traffic_map/heatmap.ipynb
/opt/nvidia/nsight-compute/2024.2.1/host/target-linux-x64/python/packages/nsys_recipe/recipes/cuda_gpu_kern_pace/pace.ipynb
/opt/nvidia/nsight-compute/2024.2.1/host/target-linux-x64/python/packages/nsys_recipe/recipes/gpu_time_util/analysis.ipynb
/opt/nvidia/nsight-compute/2024.2.1/host/target-linux-x64/python/packages/nsys_recipe/recipes/ucx_gpu_time_util_map/heatmap.ipynb
/opt/nvidia/nsight-compute/2024.2.1/host/target-linux-x64/python/packages/nsys_recipe/recipes/nvtx_gpu_proj_sum/stats.ipynb
/opt/nvidia/nsight-compute/2024.2.1/host/target-linux-x64/python/packages/nsys_recipe/recipes/nvtx_gpu_proj_sum/topN.ipynb
/opt/nvidia/nsight-compute/2024.2.1/host/target-linux-x64/python/packages/nsys_recipe/recipes/gpu_gaps/analysis.ipynb
/opt/nvidia/nsight-c

In [ ]:
%%capture
!uv pip install --force-reinstall --no-deps git+https://github.com/unslothai/unsloth-zoo
!uv pip install --force-reinstall --no-deps git+https://github.com/unslothai/unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/gpt-oss-20b-unsloth-bnb-4bit", # 20B model using bitsandbytes 4bit quantization
    "unsloth/gpt-oss-120b-unsloth-bnb-4bit",
    "unsloth/gpt-oss-20b", # 20B model using MXFP4 format
    "unsloth/gpt-oss-120b",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b-unsloth-bnb-4bit",
    dtype = None, # None for auto detection
    max_seq_length = 4096, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.9: Fast Gpt_Oss patching. Transformers: 4.56.0.dev0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gpt_oss won't work! Using float32.
Unsloth: Gpt_Oss does not support SDPA - switching to eager!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00000-of-00002.safetensors:   0%|          | 0.00/4.79G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.80G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.17G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/165 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
# Quick test to ensure model works
from transformers import TextStreamer

messages = [
    {"role": "user", "content": "Hello! Can you help me understand books?"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True,
    reasoning_effort="medium"
).to(model.device)

# Generate a test response
_ = model.generate(**inputs, max_new_tokens=256, streamer=TextStreamer(tokenizer))

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-21

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>user<|message|>Hello! Can you help me understand books?<|end|><|start|>assistant<|channel|>analysis<|message|>We need to respond politely, ask for specifics.<|end|><|start|>assistant<|channel|>final<|message|>Absolutely! I’d love to help you delve into books.  

Just let me know what you’re looking for:

- Are you exploring a specific genre (e.g., mystery, sci‑fi, historical fiction, fantasy, literary nonfiction)?
- Do you need recommendations based on a theme or a mood (e.g., “books that feel like a journey,” “something to read for a quick escape”)?
- Are you interested in a particular author and want to know what to read next?
- Do you want a deeper understanding of a certain novel—its themes, symbolism, historical context, or

In [ ]:
# Move the current notebook into the new directory
# Replace "your_notebook_name.ipynb" with the actual name of your notebook file
!mv /content/lifeline-ai-gpt-oss-20b.ipynb /content/lifeline_ai_finetuning/

mv: cannot stat '/content/lifeline-ai-gpt-oss-20b.ipynb': No such file or directory


In [ ]:
# Create a new directory
# Replace "your_folder_name" with a relevant name for your project
!mkdir /content/lifeline_ai_finetuning

In [ ]:
!ls /content/lifeline_ai_finetuning